In [1]:
import secret as k
import requests
import csv

In [2]:
#request_URL = 'http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/json/3772' # the specific location for HEATHROW
request_URL = 'http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/xml/all' # get everything. Note the JSON schema changes....
request_params = {
  'res' : 'hourly',
  'key' : k.api_key
}
response = requests.get(request_URL, request_params)
print(response.status_code)

200


In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, "html.parser")
#print(soup)

In [16]:
incoming_locations = []
incoming_observations = []
locations_to_parse = soup.find_all("location")

#locations_to_parse

for location in locations_to_parse :
  current_location_id = location.get('i')
  temp_location = {
    'continent' : location.get('continent'),
    'country' : location.get('country'),
    'elevation' : location.get('elevation'),
    'i' : current_location_id,
    'lat' : location.get('lat'),
    'lon' : location.get('lon'),
    'name' : location.get('name')
  }
  incoming_locations.append(temp_location)

  # Theoretically, this API end-point should always return at the 'Day' granularity
  for period in location.find_all("period") :
    current_day = period.get('value')

    for rep in period.find_all("rep") :
      temp_observation = {
        'LocationID' : current_location_id,
        'ObservationDate' : current_day,
        'G' : rep.get('g'),
        'T' : rep.get('t'),
        'V' : rep.get('v'),
        'D' : rep.get('d'),
        'S' : rep.get('s'),
        'W' : rep.get('w'),
        'P' : rep.get('p'),
        'Pt' : rep.get('pt'),
        'Dp' : rep.get('dp'),
        'H' : rep.get('h'),
        '$' : rep.text.strip()
      }
      incoming_observations.append(temp_observation)

#incoming_locations
#incoming_observations

In [17]:
# outputs a list of dicts to a file,
# using the keys of the first dict as the header row
# this errors if the others have additional fields
def list_to_csv(list_of_dicts, fieldnames, fname):
  with open(fname, 'w', newline='') as out_file:
    output = csv.DictWriter(out_file, fieldnames=fieldnames)

    output.writeheader()
    for row in list_of_dicts:
      output.writerow(row)


list_to_csv(incoming_locations, list(incoming_locations[0].keys()), 'incoming_locations.csv')
list_to_csv(incoming_observations, list(incoming_observations[0].keys()), 'incoming_observations.csv')